In [10]:
# IMPORT LIBRARY
import numpy as np

#######################################################################
# DEFINE QGs MAPPING
def QGs(I, rmax):
    # PREPARE IMAGE SHAPE AND CALCULATE THE NUMBER OF QUANTILES
    L1 = np.shape(I)[0] 
    L2 = np.shape(I)[1]
    Q = int(round(2*pow(L1*L2, 1.0/3.0)))
    quant = np.zeros(L1*L2, dtype = int)
    
    # CONSTRUCT VECTOR OF QUANTILES
    X = np.copy(I)
    X = np.reshape(X, (1, L1*L2))
    a = int(np.floor(L1*L2/Q))
    ind = np.zeros((1, L1*L2))
    ind[0, :] = np.arange(np.shape(X)[1])
    
    # ASSOCIATE PIXELS TO QUANTILES
    O = np.lexsort((ind, X))
    aux = np.repeat(np.arange(Q), a)
    aux = np.concatenate((aux, np.full(L1*L2 - len(aux), Q - 1)))
    quant[O] = aux
    
    # CONSTRUCT W FOR EACH VALUE OF r USING MOORE NEIGHBORHOOD
    W = np.zeros((Q, Q, rmax)) 
    i_v, j_v = np.indices((L1, L2))
    k1 = i_v * L2 + j_v
    for r in range(1, rmax + 1):
        ii_o = np.array([-r, -r, -r, 0, 0, r, r, r])
        jj_o = np.array([-r, 0, r, -r, r, -r, 0, r])
        ii = i_v[:, :, None] + ii_o
        jj = j_v[:, :, None] + jj_o
        msk = (ii >= 0) & (ii < L1) & (jj >= 0) & (jj < L2)
        ii = ii[msk]
        jj = jj[msk]
        k2 = ii * L2 + jj
        k1_new = np.repeat(k1[:, :, None], 8, axis=2)[msk]
        np.add.at(W[:, :, r - 1], (quant[k1_new], quant[k2]), 1)
        np.add.at(W[:, :, r - 1], (quant[k2], quant[k1_new]), 1)
    W = W/2
    return W
    
#######################################################################
# READ INPUT DATA
X_train = np.load("path-to-xtrain.npy")
y_train = np.load("path-to-ytrain.npy")
X_test =  np.load("path-to-xtest.npy")
y_test =  np.load("path-to-ytest.npy")

#######################################################################
# CONSTRUCT QGs FROM IMAGES USING rmax = 10
W = QGs(X_train[:, :, 0], 10)